# 🔬 Vessel Connectivity Checker - Demo Notebook

This notebook demonstrates how to use the Vessel Connectivity Checker to evaluate vessel segmentation quality using Vision-Language Models (VLMs).

## Overview

The tool:
1. Loads medical images (retinal fundus, angiograms) and their segmentation masks
2. Sends them to a VLM with specialized prompts for connectivity analysis
3. Parses the VLM response to generate structured connectivity scores
4. Visualizes results with overlays and annotations


## Setup


In [ ]:
# Add project root to path
import sys
from pathlib import Path

project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import our modules
from src.segmentation_loader import SegmentationLoader
from src.vlm_interface import VLMInterface
from src.connectivity_prompts import ConnectivityPrompts
from src.connectivity_checker import ConnectivityChecker, parse_vlm_response
from src.visualization import Visualizer, create_overlay
from src.utils import setup_sample_data, print_result

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('dark_background')

print("✓ Modules loaded successfully!")


## 1. Create Sample Data

Let's create some synthetic vessel images for demonstration. In a real scenario, you'd use DRIVE retinal images or other medical imaging datasets.


In [ ]:
# Generate synthetic sample data
samples = setup_sample_data()

print(f"\nSample files created:")
for name, path in samples.items():
    print(f"  {name}: {path}")


## 2. Load and Visualize Data


In [ ]:
# Initialize loader
loader = SegmentationLoader(target_size=(512, 512))

# Load image and masks
image = loader.load_image(samples['image'])
mask_good = loader.load_mask(samples['mask'])
mask_broken = loader.load_mask(samples['mask_broken'])

print(f"Image shape: {image.shape}")
print(f"Mask shape: {mask_good.shape}")
print(f"Broken mask shape: {mask_broken.shape}")


In [ ]:
# Visualize the data
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Original image
axes[0, 0].imshow(image)
axes[0, 0].set_title('Original Image', fontsize=12)
axes[0, 0].axis('off')

# Good mask
axes[0, 1].imshow(mask_good, cmap='gray')
axes[0, 1].set_title('Complete Segmentation', fontsize=12)
axes[0, 1].axis('off')

# Broken mask
axes[0, 2].imshow(mask_broken, cmap='gray')
axes[0, 2].set_title('Broken Segmentation', fontsize=12)
axes[0, 2].axis('off')

# Overlay on original
visualizer = Visualizer()
overlay_good = visualizer.overlay_mask(image, mask_good, alpha=0.4)
overlay_broken = visualizer.overlay_mask(image, mask_broken, alpha=0.4, color=(255, 100, 100))

axes[1, 0].imshow(overlay_good)
axes[1, 0].set_title('Overlay (Complete)', fontsize=12)
axes[1, 0].axis('off')

axes[1, 1].imshow(overlay_broken)
axes[1, 1].set_title('Overlay (Broken)', fontsize=12)
axes[1, 1].axis('off')

# Difference
diff = np.abs(mask_good.astype(int) - mask_broken.astype(int))
axes[1, 2].imshow(diff, cmap='hot')
axes[1, 2].set_title('Difference (Gaps)', fontsize=12)
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()


## 3. Explore Prompt Templates

The tool includes various prompt templates designed to analyze different aspects of vessel connectivity.


In [ ]:
# List available prompts
print("Available prompt templates:")
print("=" * 50)

for name in ConnectivityPrompts.list_prompts():
    prompt = ConnectivityPrompts.get_prompt(name)
    print(f"\n📝 {name}")
    print(f"   Category: {prompt.category}")
    print(f"   Description: {prompt.description}")
    print(f"   Keywords: {', '.join(prompt.expected_keywords[:3])}...")


In [ ]:
# View a specific prompt
prompt = ConnectivityPrompts.get_prompt('general_continuity')

print("GENERAL CONTINUITY PROMPT")
print("=" * 50)
print(prompt.template)


## 4. VLM Analysis (Requires API Key)

To run actual VLM analysis, you need an OpenAI API key:

```bash
export OPENAI_API_KEY='your-api-key'
```

Alternatively, you can use LLaVA-Med locally if you have a GPU.


In [ ]:
import os

# Check if API key is set
api_key_set = bool(os.environ.get('OPENAI_API_KEY'))
print(f"OpenAI API key configured: {'✓ Yes' if api_key_set else '✗ No'}")

if not api_key_set:
    print("\n⚠️  To run VLM analysis, set your API key:")
    print("    export OPENAI_API_KEY='sk-...'")
    print("\n📌 Continuing with simulated responses for demonstration...")


In [ ]:
# Run VLM analysis (if API key is available) or simulate response
if api_key_set:
    print("🤖 Running VLM analysis...")
    
    # Initialize VLM interface
    vlm = VLMInterface(backend='openai', model='gpt-4o-mini')
    checker = ConnectivityChecker(vlm)
    
    # Analyze the broken segmentation
    result = checker.check_connectivity(
        samples['image'],
        samples['mask_broken']
    )
    
    print_result(result.to_dict())
else:
    # Simulate a VLM response for demonstration
    simulated_response = """
    CONTINUITY: discontinuous
    BREAKS_FOUND: yes
    QUALITY_SCORE: 6
    CONFIDENCE: 0.85
    
    Analysis:
    The vessel segmentation shows some discontinuities. Specifically:
    
    BROKEN_SEGMENTS:
    1. Central main vessel - there is a visible gap in the middle section
    2. Upper branch - slight disconnection near the branch point
    
    The bifurcations generally appear intact, but the breaks in the main 
    vessel compromise the overall connectivity of the vessel tree.
    
    ANATOMICALLY_PLAUSIBLE: yes
    BIFURCATION_QUALITY: good
    """
    
    print("📝 Simulated VLM Response:")
    print(simulated_response)
    
    # Parse the simulated response
    result = parse_vlm_response(simulated_response)
    print("\n" + "=" * 50)
    print("PARSED RESULT:")
    print_result(result.to_dict())


## 5. Response Parsing Demo

Let's explore how the response parser works with different types of VLM outputs.


In [ ]:
from src.connectivity_checker import ResponseParser, check_keywords

# Test different response formats
test_responses = [
    "The vessels appear continuous throughout with no visible breaks.",
    "Multiple gaps detected: upper-left branch shows disconnection.",
    "CONTINUITY: discontinuous\nQUALITY_SCORE: 4\nCONFIDENCE: 0.78",
]

print("Response Parsing Examples")
print("=" * 60)

for i, response in enumerate(test_responses, 1):
    print(f"\n--- Example {i} ---")
    print(f"Response: {response[:60]}...")
    
    result = parse_vlm_response(response)
    print(f"\nParsed:")
    print(f"  Continuous: {result.continuous}")
    print(f"  Confidence: {result.confidence:.2f}")
    print(f"  Broken segments: {result.broken_segments}")
    
    keywords = check_keywords(response)
    print(f"\nKeyword analysis:")
    for key, value in keywords.items():
        if value:
            print(f"  ✓ {key}")


## 6. Create Analysis Report


In [ ]:
# Create a sample result for visualization
sample_result = {
    'continuous': False,
    'confidence': 0.85,
    'quality_score': 0.6,
    'broken_segments': [
        'Central main vessel - gap in middle section',
        'Upper branch - disconnection near branch point'
    ],
    'bifurcation_quality': 'good',
    'anatomically_plausible': True
}

# Create visualization
fig = visualizer.create_report_figure(
    image, 
    mask_broken, 
    sample_result,
    title="Vessel Connectivity Analysis Report"
)

if fig:
    plt.show()


## 7. Save Results


In [ ]:
from src.utils import save_json, ensure_dir, generate_report_id

# Create output directory
output_dir = ensure_dir(project_root / 'output' / 'notebook_demo')

# Save overlay image
overlay_path = output_dir / 'overlay.png'
visualizer.save_image(overlay_broken, overlay_path)
print(f"✓ Saved overlay: {overlay_path}")

# Save comparison
comparison = visualizer.create_comparison(image, mask_broken)
comparison_path = output_dir / 'comparison.png'
visualizer.save_image(comparison, comparison_path)
print(f"✓ Saved comparison: {comparison_path}")

# Save report
report_id = generate_report_id()
report_path = output_dir / f'{report_id}.json'
save_json(sample_result, report_path)
print(f"✓ Saved report: {report_path}")

# Save figure
if fig:
    fig_path = output_dir / 'analysis_figure.png'
    visualizer.save_figure(fig, fig_path)
    print(f"✓ Saved figure: {fig_path}")


## Summary

This notebook demonstrated:

1. **Data Loading**: Using `SegmentationLoader` to load and preprocess images and masks
2. **Visualization**: Creating overlays and comparison views with `Visualizer`
3. **Prompt Templates**: Exploring the different analysis prompts in `ConnectivityPrompts`
4. **VLM Analysis**: Sending images to a VLM and getting connectivity assessments
5. **Response Parsing**: Extracting structured data from VLM text responses
6. **Report Generation**: Creating and saving analysis reports

### Next Steps

- Try with real DRIVE retinal images
- Experiment with different prompt templates
- Compare results across different VLM backends (OpenAI vs LLaVA-Med)
- Extend the prompt library for specific use cases
